This notebook is used to clean and prepare the genre label data for analysis.

## Outline of Cleaning:

- [x] remove artists for which 'retrieved' value is 'none'
- [x] remove the url prefix from the retrieved artist names 
- [x] replace ' ' in the artist column with '_'
- [x] remove the '(singer)', '(rapper)', '(musician)' designation from the 'retrieved' column
- [x] remove the artists for which the retrieved-artist != searched-artist. 
    - inspect mismatches to look for typos and different versions
- [ ] Deal with genre label problems
- [ ] Normalize genre label spelling: e.g. r&b vs rhythm and blues vs rhythm & blues
    - [x] 'rock n roll' = 'rock & roll' etc, but 'rock' separate.
    - [x] remove _music
    - [x] hip hop and hip--hop -> hip-hop
    - [ ] separate lists: look at why they aren't separated
- [x] convert genre column values into lists of strings
- [x] remove bands (as opposed to individuals)
- [x] remove old columns
- [x] extract unique genres as a list
- [x] select 1% sample to verify gender

In [1]:
import numpy as np
np.random.seed(23)
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import re

from datetime import datetime

In [2]:
%ls -lt /Users/Daniel/Code/Genre/data/genre_lists/data_to_be_cleaned/

total 24032
-rw-r--r--  1 Daniel  staff  1893962 Apr 30 11:50 wiki-kaggle_genres_rescrape.csv
-rw-r--r--  1 Daniel  staff   184182 Apr 24 16:44 wiki-women_wiki_lists.csv
-rw-r--r--@ 1 Daniel  staff  1901830 Apr 23 13:41 wiki-kaggle_genres_gender.csv
-rw-r--r--@ 1 Daniel  staff  2010856 Apr 23 13:41 wiki-kaggle_genres_rough_gender.csv
-rw-r--r--@ 1 Daniel  staff   708936 Apr 22 15:30 wiki-kaggle_genres_rough.csv
-rw-r--r--  1 Daniel  staff   895886 Apr 17 10:27 kaggle_genres-reduced.csv
-rwxr-xr-x@ 1 Daniel  staff    22397 Apr 15 13:24 wiki_bands_women.csv*
-rwxr-xr-x@ 1 Daniel  staff    37575 Apr 15 13:24 wiki_country_women.csv*
-rwxr-xr-x@ 1 Daniel  staff    79119 Apr 15 13:24 wiki_rock_women.csv*
-rw-r--r--@ 1 Daniel  staff  1949982 Apr 15 11:34 kaggle_genres_rough.csv
-rwxr-xr-x@ 1 Daniel  staff    15444 Apr 11 12:41 wiki_bands_women-cleaned.csv*
-rwxr-xr-x@ 1 Daniel  staff    24278 Apr 11 12:41 wiki_country_women-cleaned.csv*
-rwxr-xr-x@ 1 Daniel  staff    53345 Apr 11 12:41 wiki_r

### Data Sets

The file singers_gender.csv is from Kaggle and lists music artists and their gender. This is our starting point. It is augmented using the lists of women artists. Genre and network info will be generated by scraping databases. For now we are focusing in Wikipedia.

In [3]:
# kaggle_data = pd.read_csv('singers_gender.csv', encoding = 'latin-1')

In [4]:
# kaggle_data.shape

### Load the data to be cleaned:

Current: wiki-kaggle_genres_rough.csv

- This will be replaced by the fully scraped set
- The full set needs to be cleaned

Add in a converter that splits the genre list on commas:
https://stackoverflow.com/questions/32742976/how-to-read-a-column-of-csv-as-dtype-list-using-pandas

In [5]:
data = pd.read_csv('../../data/genre_lists/data_to_be_cleaned/wiki-kaggle_genres_rescrape.csv', header = 0)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [6]:
data.head(50)

,artist,gender,retrieved,genre
0,Jimmy Boyd,male,none,none
1,Christopher Willits,male,https://en.wikipedia.org/wiki/Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac..."
2,Henry Frayne,male,none,none
3,Shawn Hook,male,https://en.wikipedia.org/wiki/Shawn_Hook,"['pop', 'electronic', 'rock']"
4,Steve Poltz,male,https://en.wikipedia.org/wiki/Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']"
5,Dave Schramm,male,none,none
6,Marvin Isley,male,https://en.wikipedia.org/wiki/Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']"
7,Povel Ramel,male,https://en.wikipedia.org/wiki/Povel_Ramel,['vaudeville']
8,Lisa McHugh,female,none,none
9,Sharon Van Etten,female,https://en.wikipedia.org/wiki/Sharon_Van_Etten,"['indie rock', 'indie folk']"


In [7]:
data.shape

(23177, 4)

In [8]:
data.isnull().sum()

artist       0
gender       0
retrieved    0
genre        0
dtype: int64

For how many artists is the scraped genre 'none':

In [9]:
(data.genre == 'none').sum()

7677

For how many artists is the 'retrieved' value 'none':

In [10]:
(data.retrieved == 'none').sum()

7677

Take a glance at artist and retrieved values to determine necessary cleaning:

In [11]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

for n in rints:
    print('artist: {}        retrieved: {}'.format(data.artist.iloc[n], data.retrieved.iloc[n]))

artist: Jeanne Cherhal        retrieved: https://en.wikipedia.org/wiki/Jeanne_Cherhal
artist: Jessica Cauffiel        retrieved: none
artist: Marco Marinangeli        retrieved: https://en.wikipedia.org/wiki/Marco_Marinangeli
artist: Rosemary Clooney        retrieved: https://en.wikipedia.org/wiki/Rosemary_Clooney
artist: Jon Rennard        retrieved: none
artist: Colm Wilkinson        retrieved: https://en.wikipedia.org/wiki/Colm_Wilkinson
artist: Dwight Pinkney        retrieved: https://en.wikipedia.org/wiki/Dwight_Pinkney
artist: Nivea        retrieved: https://en.wikipedia.org/wiki/Nivea_(singer)
artist: Emilie Hammarskjöld        retrieved: none
artist: J. D. Souther        retrieved: https://en.wikipedia.org/wiki/J._D._Souther
artist: Øyvind Nypan        retrieved: none
artist: Brownstone        retrieved: https://en.wikipedia.org/wiki/Brownstone_(band)
artist: Frankie Armstrong        retrieved: https://en.wikipedia.org/wiki/Frankie_Armstrong
artist: Chuck Berghofer        retri

Notes on Retrieved:

- Underscore is used to separate parts of the name
- '.' are allowed in names 
- '(singer)' and '(musician)' are sometimes included and need to be stripped (probably to distinguish from othe people in wikipedia)
- double quotes are allowed in names
- hyphens appear

### Remove artists for which 'retrieved' value is 'none'

Convert none to null:

In [12]:
data['retrieved'] = data['retrieved'].replace('none', np.nan)

In [13]:
data.head()

,artist,gender,retrieved,genre
0,Jimmy Boyd,male,NaN,none
1,Christopher Willits,male,https://en.wikipedia.org/wiki/Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac..."
2,Henry Frayne,male,NaN,none
3,Shawn Hook,male,https://en.wikipedia.org/wiki/Shawn_Hook,"['pop', 'electronic', 'rock']"
4,Steve Poltz,male,https://en.wikipedia.org/wiki/Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']"


In [14]:
data.isnull().sum()

artist          0
gender          0
retrieved    7677
genre           0
dtype: int64

Drop rows with nulls:

In [15]:
data.dropna(axis = 0, inplace = True)

In [16]:
data.shape

(15500, 4)

In [17]:
data.head()

,artist,gender,retrieved,genre
1,Christopher Willits,male,https://en.wikipedia.org/wiki/Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac..."
3,Shawn Hook,male,https://en.wikipedia.org/wiki/Shawn_Hook,"['pop', 'electronic', 'rock']"
4,Steve Poltz,male,https://en.wikipedia.org/wiki/Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']"
6,Marvin Isley,male,https://en.wikipedia.org/wiki/Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']"
7,Povel Ramel,male,https://en.wikipedia.org/wiki/Povel_Ramel,['vaudeville']


## Remove the prefix from the 'retrieved' values

In [18]:
"""This function extracts artist name from the url.
Apply it to the 'retrieved' values."""
def retrieved_artist(text):
    try:
        retrieved = text
        p = re.compile(r'(https://en.wikipedia.org/wiki/)(.*)')
        result = re.match(p, retrieved)
        return result.group(2)
    except:
        if text == 'none':
            return 'none'
    else:
        return 'None'

In [19]:
data['retrieved'] = data.retrieved.apply(retrieved_artist)

In [20]:
data.head()

,artist,gender,retrieved,genre
1,Christopher Willits,male,Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac..."
3,Shawn Hook,male,Shawn_Hook,"['pop', 'electronic', 'rock']"
4,Steve Poltz,male,Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']"
6,Marvin Isley,male,Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']"
7,Povel Ramel,male,Povel_Ramel,['vaudeville']


Take a glance at artist and retrieved values after cleaning retrieved to determine further cleaning:

In [21]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

for n in rints:
    print('artist: {}        retrieved: {}'.format(data.artist.iloc[n], data.retrieved.iloc[n]))

artist: Dominique Eade        retrieved: Dominique_Eade
artist: John C. J. Taylor        retrieved: John_C._J._Taylor
artist: Phillip Mitchell        retrieved: Phillip_Mitchell
artist: Meja        retrieved: Meja
artist: Simon Underwood        retrieved: Simon_Underwood
artist: Del Reeves        retrieved: Del_Reeves
artist: Syleena Johnson        retrieved: Syleena_Johnson
artist: Jimi Jamison        retrieved: Jimi_Jamison
artist: John Spiker        retrieved: John_Spiker
artist: Melissa Sgambelluri        retrieved: Melissa_Sgambelluri
artist: Justin Hayford        retrieved: Justin_Hayford
artist: Markus Fagervall        retrieved: Markus_Fagervall
artist: John Schlitt        retrieved: John_Schlitt
artist: Jack Hues        retrieved: Jack_Hues
artist: Paris Bennett        retrieved: Paris_Bennett


## Replace spaces with _ in the artist column:

In [22]:
"""This function replaces white space in the values of
the column artist with an underscore."""
def underscore(text):
    try:
        split_name = text.split(' ')
        return '_'.join(split_name)  
    except:
        return 'error'

In [23]:
data['artist'] = data.artist.apply(underscore)

In [24]:
data.head()

,artist,gender,retrieved,genre
1,Christopher_Willits,male,Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac..."
3,Shawn_Hook,male,Shawn_Hook,"['pop', 'electronic', 'rock']"
4,Steve_Poltz,male,Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']"
6,Marvin_Isley,male,Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']"
7,Povel_Ramel,male,Povel_Ramel,['vaudeville']


## Remove the \_(singer) type designation from retrieved

In [25]:
"""This function uses re. to remove any parenthetical designations
form the retrieved artist name"""
def remove_designation(text):
    designations = [r'_\(singer\)', r'_\(musician\)', r'_\(rapper\)', r'_\(band\)', r'_\(composer\)', r'_\(music_producer\)', r'_\(singer-songwriter\)' ]
    x = text
    for des in designations:
        if re.search(des, x):
            x = re.sub(r'{}'.format(des),'',text)
    return x

Apply the function:

In [26]:
data['retrieved_clean'] = data.retrieved.apply(remove_designation)

Take a glance at artist and retrieved_clean values:

In [27]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

data[['artist','retrieved_clean']].iloc[rints]

,artist,retrieved_clean
3281,Julian_Velard,Julian_Velard
9987,Chingy,Chingy
12032,Deesha,Deesha
9981,Jamie_Oldaker,Jamie_Oldaker
6919,Pearl_Future,Pearl_Future
10485,Vince_Taylor,Vince_Taylor
15091,Tarsame_Singh_Saini,Tarsame_Singh_Saini
16312,Bradford_Cox,Bradford_Cox
3036,Chubby_Checker,Chubby_Checker
12532,Charlie_Dominici,Charlie_Dominici


### Mark the rows for which retrieved_clean is different from artist

In [28]:
"""This function takes a pair of strings and checks
if they are equivalent (case insensitive)

.casefold is used to be case insensitive; 
still might have problems on some characters"""

def verify_artist(x,y):
    if x.casefold() == y.casefold(): 
        return 1
    else:
        return 0

Introduce a mismatch just to make sure we can properly remove these:

In [29]:
# use an iloc index larger than the size of the original dataframe
#data.iloc[data.shape[0]+1] = ['test','test_wrong','universal','test_wrong']

Apply the function:

In [30]:
data['match'] = (data.artist.apply(lambda x: x.casefold()) != data.retrieved_clean.apply(lambda x: x.casefold())).astype('int64')

In [31]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
1,Christopher_Willits,male,Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac...",Christopher_Willits,0
3,Shawn_Hook,male,Shawn_Hook,"['pop', 'electronic', 'rock']",Shawn_Hook,0
4,Steve_Poltz,male,Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']",Steve_Poltz,0
6,Marvin_Isley,male,Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']",Marvin_Isley,0
7,Povel_Ramel,male,Povel_Ramel,['vaudeville'],Povel_Ramel,0


In [32]:
data.match.sum()

0

Now remove artists where retrieved_clean doesn't match artist:

In [33]:
data = data[data.match == 0]

In [34]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
1,Christopher_Willits,male,Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac...",Christopher_Willits,0
3,Shawn_Hook,male,Shawn_Hook,"['pop', 'electronic', 'rock']",Shawn_Hook,0
4,Steve_Poltz,male,Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']",Steve_Poltz,0
6,Marvin_Isley,male,Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']",Marvin_Isley,0
7,Povel_Ramel,male,Povel_Ramel,['vaudeville'],Povel_Ramel,0


Now the remaining artists are verified and have non-null genre label. 

In [35]:
data = data.copy( deep = True)

### Genre Labels

Each value of the genre column is a _string_ of comma separated genre labels using the spotify abbreviations. We want to convert it to a _list_ of strings.

# Problems! 

Strings to search for to possibly correct:

- [ ] 'and' -- too many!
- [x] '\'
- [x] 'r&g' -- OK - rhythm and grime
- [x] '*'
- [x] '_·_' -- added genrelist function; replaced with comma
- [ ] search for all non-letters
- [x] 'descriptors'- '|'
- [x] 'rock_folk_-rock_rock_-electronic_ballad'
- [x] 'hillbilly_rockabilly_r&b'
- [x] '-' at the beggining of a string
- [x] '\xa0' (no break space)

More examples:

- [x] 'hardcore\\\\xa0punk'
- [x] 'country_·_americana_·_folk_·_singer_songwriter'
- [x] 'blues_soul_r_&_b_gospel_funk_folk'
- [x] 'college\\\\xa0rock'
- [ ] 'gospel_and_gospel_blues'

Clearly some of these need to be parsed into common genres. 

In [36]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
1,Christopher_Willits,male,Christopher_Willits,"['electronic', 'glitch', 'ambient', 'electroac...",Christopher_Willits,0
3,Shawn_Hook,male,Shawn_Hook,"['pop', 'electronic', 'rock']",Shawn_Hook,0
4,Steve_Poltz,male,Steve_Poltz,"['pop rock', 'indie-rock', 'folk rock']",Steve_Poltz,0
6,Marvin_Isley,male,Marvin_Isley,"['r&b', 'funk', 'soul', 'funk rock']",Marvin_Isley,0
7,Povel_Ramel,male,Povel_Ramel,['vaudeville'],Povel_Ramel,0


In [37]:
data.genre.isnull().sum()

0

## Deal with unusual cases by hand:

In [38]:
data.loc[1666]

artist                       Sophie_B._Hawkins
gender                                  female
retrieved                    Sophie_B._Hawkins
genre              ['rock', 'pop', 'pop rock']
retrieved_clean              Sophie_B._Hawkins
match                                        0
Name: 1666, dtype: object

Remove any '*'

In [39]:
data.genre[data.genre.str.contains(r'\*')]

18410    ['zouk', '*', 'r&b']
Name: genre, dtype: object

In [40]:
data.genre.at[18410] = "['zouk', 'r&b']" # need to make it a string so it is as same type as rest of column

In [41]:
data[data.genre.str.contains(r'[*]')] 

,artist,gender,retrieved,genre,retrieved_clean,match


In [42]:
data.isnull().sum()

artist             0
gender             0
retrieved          0
genre              0
retrieved_clean    0
match              0
dtype: int64

In [43]:
data[data.genre.str.contains(r'descriptors\\')]

,artist,gender,retrieved,genre,retrieved_clean,match
6650,RY_X,male,RY_X,"['ambient pop', 'folktronica', 'indietronica',...",RY_X,0


In [44]:
data.genre.at[6650] = "['ambient pop', 'folktronica', 'indietronica', 'indie folk']"

In [45]:
data[data.genre.str.contains(r'\\u2060')]

,artist,gender,retrieved,genre,retrieved_clean,match
17600,Ben_Moody,male,Ben_Moody,"['alternative rock', 'alternative metal', '\u2...",Ben_Moody,0


In [46]:
data.genre.loc[17600]

"['alternative rock', 'alternative metal', '\\u2060', 'nu metal', 'gothic metal']"

In [47]:
data.genre.at[17600] = "['alternative rock', 'alternative metal', 'nu metal', 'gothic metal']"

In [48]:
data[data.genre.str.contains(r'\\')]

,artist,gender,retrieved,genre,retrieved_clean,match
646,Gabriel_Wilson,male,Gabriel_Wilson,"['christian & gospel', 'independent\nsinger/so...",Gabriel_Wilson,0
1306,Eric_Gaffney,male,Eric_Gaffney,"['alternative rock', 'indie rock', 'lo-fi', 'h...",Eric_Gaffney,0
3015,Michael_Stipe,male,Michael_Stipe,"['alternative\xa0rock', 'folk rock', 'college\...",Michael_Stipe,0
8147,John_Wesley_Harding,male,John_Wesley_Harding_(singer),"['singer-songwriter rock\npop', 'folk']",John_Wesley_Harding,0
9899,Marcus_Singletary,male,Marcus_Singletary,"['rock', 'progressive\xa0rock', 'jazz', 'jazz ...",Marcus_Singletary,0
16275,Patrick_Dennis,male,Patrick_Dennis_(musician),"['alternative rock', 'indie rock', 'alternativ...",Patrick_Dennis,0
16726,Jamal_Millner,male,Jamal_Millner,['black american music \nwestern european art ...,Jamal_Millner,0
19018,Lance_King,male,Lance_King,"['heavy metal', 'power metal', 'progressive me...",Lance_King,0


In [49]:
data[data.genre.str.contains(r' -')]

,artist,gender,retrieved,genre,retrieved_clean,match
11961,Sarana_VerLin,female,Sarana_VerLin,"['celticana - a blend of rock', 'pop', 'folk',...",Sarana_VerLin,0
13278,Jane_Castro,female,Jane_Castro,"['pop', ' house music - latin american']",Jane_Castro,0
15232,Declan_Galbraith,male,Declan_Galbraith,"['pop', 'rock folk -rock rock -electronic ball...",Declan_Galbraith,0


In [50]:
data.genre.loc[11961]

"['celticana - a blend of rock', 'pop', 'folk', 'celtic and americana']"

In [51]:
data.genre.at[11961] = "['celticana']"

In [52]:
data.genre.loc[13278]

"['pop', ' house music - latin american']"

In [53]:
data.genre.at[13278] = "['pop', 'house music']"

In [54]:
data.genre.loc[15232]

"['pop', 'rock folk -rock rock -electronic ballad']"

In [55]:
data.genre.at[15232] = "['pop', 'rock', 'folk-rock', rock-electronic, ballad']"

In [56]:
data[data.genre.str.contains(r'hillbilly')]

,artist,gender,retrieved,genre,retrieved_clean,match
6897,Webb_Pierce,male,Webb_Pierce,"['country', 'honky-tonk', 'western swing', 'co...",Webb_Pierce,0
7743,Boyden_Carpenter,male,Boyden_Carpenter,"['bluegrass', 'bluegrass gospel', 'hillbilly']",Boyden_Carpenter,0
8854,Harvie_June_Van,female,Harvie_June_Van,"['country', 'hillbilly']",Harvie_June_Van,0
15556,Bill_Holford,male,Bill_Holford,"['cajun', 'country', ' hillbilly rockabilly r&b']",Bill_Holford,0


In [57]:
data.genre.loc[15556]

"['cajun', 'country', ' hillbilly rockabilly r&b']"

In [58]:
data.genre.at[15556] = "['cajun', 'country', ' hillbilly, rockabilly, r&b']"

In [59]:
data[data.genre.str.contains(r'r & b')]

,artist,gender,retrieved,genre,retrieved_clean,match
2599,Larissa_Lam,female,Larissa_Lam,"['dance-pop', 'r & b', 'jazz']",Larissa_Lam,0
6853,Nick_Clemons,male,Nick_Clemons,"['rock', 'funk', 'r & b']",Nick_Clemons,0
14153,A._J._Ghent,male,A._J._Ghent,"['funk', 'soul', 'jam', 'gospel', 'sacred stee...",A._J._Ghent,0
14654,Thomasina_Winslow,female,Thomasina_Winslow,"['blues soul r & b gospel funk folk', 'african...",Thomasina_Winslow,0
16319,Jeff_Pehrson,male,Jeff_Pehrson,"['rock', 'psychedelic rock', 'r & b', 'folk ro...",Jeff_Pehrson,0


In [60]:
data.genre[data.genre.str.contains('active')]

11935    ['rock', " active rock country jazz standards ...
Name: genre, dtype: object

In [61]:
data.genre.loc[11935]

'[\'rock\', " active rock country jazz standards children\'s folk"]'

In [62]:
data.genre.at[11935] = '[rock, active rock, country, jazz standards, childrens folk]'

In [63]:
data.genre[data.genre.str.contains('pitbash')]

13009    ['punk rock', 'art rock', 'jewish rock', 'gara...
Name: genre, dtype: object

In [64]:
data.genre.loc[13009]

'[\'punk rock\', \'art rock\', \'jewish rock\', \'garage punk\', \'obscuro\', \'metal\', \'marching band\', \'jewish music\', \'jazz\', \' pitbash jewish punk "thrash opera"\']'

In [65]:
data.genre.at[11935] = "['punk rock', 'art rock', 'jewish rock', 'garage punk', 'obscuro', 'metal','marching band', 'jewish music', 'jazz',  'pitbash', 'jewish punk', 'thrash', 'opera']"

Remove 'earlier:'

In [66]:
data.genre.loc[11170]

"['aor', 'pop rock', '; earlier:', 'pop', 'disco', 'soul']"

In [67]:
data.at[11170,'genre'] = "['aor', 'pop_rock', 'pop', 'disco', 'soul']"

In [68]:
pd.set_option('display.max_rows', None)
pd.options.display.max_rows
data.genre[data.genre.str.contains(r"\balt\b")]

276                             ['alt-rock', 'indie rock']
366                                           ['alt rock']
523      ['pop rock', 'alt rock', 'third wave ska', 'fi...
1491                  ['folk', 'americana', 'alt-country']
1514                       ['country rock', 'alt-country']
1740                      ['folk', 'roots', 'alt-country']
2121     ['americana', 'alt country', 'folk', 'rock', '...
2487     ['americana', 'folk', 'folk rock', 'country fo...
2540        ['rock', 'hard rock', 'alt rock', 'punk rock']
3214                                       ['alt-country']
3369                                          ['alt-folk']
3558                                    ['pop', 'alt-pop']
4014                                ['pop', 'alt-country']
4043     ['honky tonk', 'country', 'americana', 'alt. c...
4175                               ['folk', 'alt-country']
4210               ['folk', 'alt country', 'country rock']
4449     ['rock', 'acoustic', 'alt rock', 'pop', 'film .

## Spelling normalizations:

Make genre label spelling replacements: (and, &, n, 'n, 'n')

- rock and roll
- rhythm and blues
- rhythm and grime
- country and western
- RnB

- [ ] Tom will supply further normalizations

In [80]:
"""This function takes normalizes genre spellings."""
def genrespelling(string):
    string = string.replace(r'r & b','r&b') \
    .replace(r'rock and roll','rock&roll') \
    .replace(r'rock & roll','rock&roll') \
    .replace(r'rock-&-roll','rock&roll') \
    .replace(r'rock n roll','rock&roll') \
    .replace(r'rock-n-roll','rock&roll') \
    .replace(r'rock-and-roll','rock&roll') \
    .replace(r"rock 'n roll",'rock&roll') \
    .replace(r'country and western','country&western') \
    .replace(r'rhythm and grime','r&g') \
    .replace(r'','') \
    .replace("electronic dance music", "edm")
    string = re.sub(r"r.{0,1}n.{0,1}b","r&b", string)
    string = re.sub(r"hip.{0,1}hop","hip-hop", string)
    string = re.sub(r"adult","", string)
    string = re.sub(r"afrobeats","afrobeat", string)
    string = re.sub(r"boleros","bolero", string)
    string = re.sub(r"musicals","musical", string)
    string = re.sub(r"neo_souls","neo_soul", string)
    string = re.sub(r"protest_songs","protest_song", string)
    string = re.sub(r"spirituals","spiritual", string)
    string = re.sub(r"television_scores","television_score", string)
    string = re.sub(r"show_tune","show_tunes", string)
    string = re.sub(r"ballad","ballads", string)
    string = re.sub(r"soundtracks","soundtrack", string)
    string = re.sub(r"afropop","afro-pop", string)
    string = re.sub(r"alt/rock","alternative-rock", string)
    string = re.sub(r"alt. {0,1}country","alternative-country", string)
    string = re.sub(r"\balt-","alternative-", string)
    string = re.sub(r"\balt\b","alternative", string)
    string = re.sub(r"alternative ","alternative-", string)
    return string

In [81]:
data['genre_respell']= data['genre'].apply(genrespelling)

In [82]:
data.genre[data.genre_respell.str.contains(r"\balternative\b")].shape, data.genre[data.genre_respell.str.contains("alternative-")].shape

((1770,), (1612,))

In [83]:
data.genre[data.genre.str.contains(r"alternative[^- ']")]

3015     ['alternative\xa0rock', 'folk rock', 'college\...
16275    ['alternative rock', 'indie rock', 'alternativ...
Name: genre, dtype: object

In [84]:
data.genre_respell[data.genre_respell.str.contains(r"alternative[^- ']")]

3015     ['alternative\xa0rock', 'folk rock', 'college\...
16275    ['alternative-rock', 'indie rock', 'alternativ...
Name: genre_respell, dtype: object

In [85]:
data.genre_respell.loc[4043]

"['honky tonk', 'country', 'americana', 'alternative-country']"

In [86]:
data.genre.loc[4043]

"['honky tonk', 'country', 'americana', 'alt. country']"

## General String Cleaning

The following function will systematically reformat the genre lists to a great extent. After applying it, there are still a few cases that are dealt with individually and some that are dropped.

In [90]:
"""This function takes in a string of the form
appearing in the genrelist of the dataframe.
It strips the square brackets and extra quotes and
returns a list of strings where each string is a genre label.
It also removes strings in parentheses and removes \( or \) that are isolated.
It replaces 'singer/songwriter' with 'singer-songwriter' and replaces forward slashes with commas."""

def genrelist(string):
    string = string.strip("[").strip("]").replace("'","").replace('"',"") \
    .replace("singer/songwriter", "singer-songwriter").replace("/",",").replace("·",",") \
    .replace(r";",",").replace(r"|",",").replace(u"\xa0",u" ")\
    .replace(u"\\xa0",u" ")\
    .replace(r"\n",",")
    L = [s for s in string.split(',')]
    L_new = []
    for x in L:
        x = re.sub(r"\(.*?\)", "", x) 
        x = re.sub(r"\(", "", x) 
        x = re.sub(r"\)", "", x) 
        x = re.sub(r":", "", x)
        x = re.sub(r"\.", "", x)
        x = re.sub(r"\]", "", x)
        x = re.sub(r"\[", "", x)
        x = x.replace(" ","_").lstrip("_").rstrip("_").lstrip("-").rstrip("-")
        x = re.sub(r"\band_", "", x)
        x = re.sub(r"_music\b", "", x)
        x = re.sub(r"-", "_", x)
        L_new.append(x)
    while (str("") in L_new):
        L_new.remove("")
    return L_new

# old version
# def genrelist(string):
#     string = string.strip("[").strip("]").replace("'","").replace('"',"") \
#     .replace("singer/songwriter", "singer-songwriter").replace("/",",").replace("·",",") \
#     .replace(r"\n",",").replace(r";",",").replace(r"|",",").replace(u"\xa0",u" ")\
#     .replace(u"\\xa0",u" ")
#     L = [s for s in string.split(',')]
#     L_new = []
#     for x in L:
#         x = re.sub(r"\(.*?\)", "", x) # newly introduced
#         x = re.sub(r"\(", "", x) # newly introduced
#         x = re.sub(r"\)", "", x) # newly introduced
#         x = x.replace(" ","_").lstrip("_").rstrip("_").lstrip("-").lstrip(r"and_").rstrip(r"_music") #removed .strip(";") and added others
#         L_new.append(x)
#     while (str("") in L_new):
#         L_new.remove("")
#     return L_new

In [91]:
data['genrelist']= data['genre'].apply(genrelist)

In [96]:
data.genrelist[data.genre.str.contains(r"alternative[^- ']")]

3015     [alternative_rock, folk_rock, college_rock, ja...
16275    [alternative_rock, indie_rock, alternative_cou...
Name: genrelist, dtype: object

We can still use 'genre' column for str.contains searches.

There are a few more issues to fix by hand:

Two entries with .mw-parser... -- Fix by hand.

In [74]:
data.genrelist[data.genre[data.genre.str.contains(r'\.mw-.*')].index]

2416    [.mw-parser-output_div.columns-2_div.column{fl...
7861    [.mw-parser-output_div.columns-2_div.column{fl...
Name: genrelist, dtype: object

In [75]:
data.genrelist.loc[2416]

['.mw-parser-output_div.columns-2_div.column{floatleft',
 'width50%',
 'min-width300px}.mw-parser-output_div.columns-3_div.column{floatleft',
 'width33.3%',
 'min-width200px}.mw-parser-output_div.columns-4_div.column{floatleft',
 'width25%',
 'min-width150px}.mw-parser-output_div.columns-5_div.column{floatleft',
 'width20%',
 'min-width120px}',
 'disco',
 'funk',
 'electric',
 'latin_soul']

In [76]:
data.at[2416,'genrelist'] = ['disco', 'funk', 'electric', 'latin_soul']

In [77]:
data.genrelist.loc[7861]

['.mw-parser-output_div.columns-2_div.column{floatleft',
 'width50%',
 'min-width300px}.mw-parser-output_div.columns-3_div.column{floatleft',
 'width33.3%',
 'min-width200px}.mw-parser-output_div.columns-4_div.column{floatleft',
 'width25%',
 'min-width150px}.mw-parser-output_div.columns-5_div.column{floatleft',
 'width20%',
 'min-width120px}',
 'torch_song']

In [78]:
data.at[7861,'genrelist'] = ['torch_song']

Let's look at the list of genre labels that we have just dealt with in detail:

In [79]:
trouble_index = data.genre[data.genre.str.contains(r'[;)(/\\]')].index

In [80]:
trouble = data.loc[trouble_index]

In [81]:
trouble.head()

,artist,gender,retrieved,genre,retrieved_clean,match,genrelist
254,Eddy_Oh,male,Eddy_Oh,"['k-pop', ';', 'hip-hop', ';', 'dance']",Eddy_Oh,0,"[k-pop, hip-hop, dance]"
331,Jillian_Wheeler,female,Jillian_Wheeler,['indie rock/pop/electronic'],Jillian_Wheeler,0,"[indie_rock, pop, electronic]"
349,Caleb_Shomo,male,Caleb_Shomo,"['metalcore', 'electronic', 'electropop', 'har...",Caleb_Shomo,0,"[metalcore, electronic, electropop, hardcore_p..."
503,Amy_Black,female,Amy_Black_(singer),['roots/blues/soul'],Amy_Black,0,"[roots, blues, soul]"
646,Gabriel_Wilson,male,Gabriel_Wilson,"['christian & gospel', 'independent\nsinger/so...",Gabriel_Wilson,0,"[christian_&_gospel, independent, singer-songw..."


In [82]:
trouble_genre_list = trouble.genrelist.values.tolist()
trouble_genre_list = [x for y in trouble_genre_list for x in y]
trouble_genre_list = list(set(trouble_genre_list))

In [83]:
len(trouble_genre_list)

218

We find that 'era' appears and explore further. Find that one instance exists and we remove it.

In [84]:
era = trouble.genre[trouble.genre.str.contains(r'\bera')]

In [85]:
era

7908    ['rock', 'alternative rock', 'experimental', '...
Name: genre, dtype: object

In [86]:
data.genrelist.loc[7908]

['rock',
 'alternative_rock',
 'experimental',
 'mpb',
 'progressive_rock',
 'post-punk',
 'new_wave',
 'samba_rock',
 'cuidado!',
 'era']

In [87]:
data.at[7908,'genrelist'] = ['rock',
 'alternative_rock',
 'experimental',
 'mpb',
 'progressive_rock',
 'post-punk',
 'new_wave',
 'samba_rock',
 ]

In [88]:
data.genrelist.loc[7908]

['rock',
 'alternative_rock',
 'experimental',
 'mpb',
 'progressive_rock',
 'post-punk',
 'new_wave',
 'samba_rock']

We look through the list of genres that were causing trouble due to special characters:

In [89]:
n = 8
trouble_genre_list[25*n:25*(n+1)]

['synth-pop',
 'acid_rock',
 'americana',
 'abakuá',
 'pop-classical',
 'rock&roll',
 'operatic_pop',
 'roots',
 'gulf_and_western',
 'folkloric',
 'film',
 'acoustic',
 'classical',
 'soundtrack',
 'aor',
 'electropop',
 'garage_punk',
 'folk-pop']

In [90]:
nu = data.genrelist[data.genre.str.contains(r'\bnu\b')]

In [91]:
nu.iloc[6]

['dansband', 'pop', 'nu-disco', 'schlager']

- [ ] Check that all of the problematic symbols are removed.

- [ ] Search for labels with two or more "_" as these might be multiple labels stuck together

In [92]:
data.shape

(15500, 7)

### Remove all artists with null values for genre :

In [93]:
data.isnull().sum(axis = 0)

artist             0
gender             0
retrieved          0
genre              0
retrieved_clean    0
match              0
genrelist          0
dtype: int64

In [94]:
data = data[data['genrelist'].notnull()]

In [95]:
data.isnull().sum(axis = 0)

artist             0
gender             0
retrieved          0
genre              0
retrieved_clean    0
match              0
genrelist          0
dtype: int64

In [96]:
data.shape

(15500, 7)

### Create column with length of genre lists:

In [97]:
data['genrelist_length'] = data.genrelist.apply(lambda x: len(x))

In [98]:
data.shape

(15500, 8)

### Remove artists for which the genre list is empty:

In [99]:
data[data.genrelist_length == 0]

,artist,gender,retrieved,genre,retrieved_clean,match,genrelist,genrelist_length
4024,Cupid,male,Cupid_(singer),[],Cupid,0,[],0
15255,Mary_Zilba,female,Mary_Zilba,[],Mary_Zilba,0,[],0
17487,Betty_Clooney,female,Betty_Clooney,[],Betty_Clooney,0,[],0


Note: Cupid, Mary Zilba, Betty Clooney have made it to this point but actually have no genres listed in wikipedia.

In [100]:
data = data[data.genrelist_length > 0]

In [101]:
data.shape

(15497, 8)

### Remove bands

In [102]:
data.shape, data[~data.retrieved.str.contains('band')].shape, data[data.retrieved.str.contains('band')].shape

((15497, 8), (15471, 8), (26, 8))

In [103]:
data = data[~data.retrieved.str.contains('band')].copy(deep = True)

In [104]:
data.shape

(15471, 8)

Remove old columns:

In [105]:
data.columns

Index(['artist', 'gender', 'retrieved', 'genre', 'retrieved_clean', 'match',
       'genrelist', 'genrelist_length'],
      dtype='object')

In [106]:
data.drop(['retrieved','genre','retrieved_clean', 'match'], axis = 1, inplace = True)

In [107]:
data.head()

,artist,gender,genrelist,genrelist_length
1,Christopher_Willits,male,"[electronic, glitch, ambient, electroacoustic,...",5
3,Shawn_Hook,male,"[pop, electronic, rock]",3
4,Steve_Poltz,male,"[pop_rock, indie-rock, folk_rock]",3
6,Marvin_Isley,male,"[r&b, funk, soul, funk_rock]",4
7,Povel_Ramel,male,[vaudeville],1


In [108]:
data.shape

(15471, 4)

## Gender

Remove any artists without gender value

In [109]:
data.gender.isnull().sum()

0

In [110]:
data.gender.unique()

array(['male', 'female'], dtype=object)

### Extracting the unique genre labels:

In [111]:
genre_list = data.genrelist.values.tolist()
genre_list = [x for y in genre_list for x in y]
genre_list = list(set(genre_list))

In [112]:
genre_list[:5]

['progressive_jazz', 'drone_rock', 'jazz_fusion', 'thrash', 'retro']

In [113]:
len(genre_list)

1717

In [114]:
print('There are {} artists with genre and binary-gender labels.'.format(data.shape[0]))
print('There are {} unique genre labels.'.format(len(genre_list)))

There are 15471 artists with genre and binary-gender labels.
There are 1717 unique genre labels.


Check to see if there are artists with len(genrelist) = 0

In [115]:
data[data.genrelist.apply(lambda x: True if len(x) == 0 else False)]

,artist,gender,genrelist,genrelist_length


In [116]:
data.head()

,artist,gender,genrelist,genrelist_length
1,Christopher_Willits,male,"[electronic, glitch, ambient, electroacoustic,...",5
3,Shawn_Hook,male,"[pop, electronic, rock]",3
4,Steve_Poltz,male,"[pop_rock, indie-rock, folk_rock]",3
6,Marvin_Isley,male,"[r&b, funk, soul, funk_rock]",4
7,Povel_Ramel,male,[vaudeville],1


### Export full data set for further use:

In [117]:
today = datetime.today()
now = today.strftime('%Y-%m-%d-%H-%M')
#now = 'temp'

In [118]:
data.to_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_genres_gender_cleaned_{}.csv'.format(now))

### Export the list of genres:

In [119]:
genre_list_df = pd.DataFrame({'genre_list':genre_list})

In [120]:
genre_list_df.to_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/genre_list_{}.csv'.format(now))

In [121]:
%ls -lt ../../data/genre_lists/data_ready_for_model/

total 21624
-rw-r--r--  1 Daniel  staff   28796 May  5 14:03 genre_list_2020-05-05-14-03.csv
-rw-r--r--  1 Daniel  staff  954255 May  5 14:03 wiki-kaggle_genres_gender_cleaned_2020-05-05-14-03.csv
-rw-r--r--  1 Daniel  staff   28828 May  5 14:00 genre_list_2020-05-05-14-00.csv
-rw-r--r--  1 Daniel  staff  954385 May  5 14:00 wiki-kaggle_genres_gender_cleaned_2020-05-05-14-00.csv
-rw-r--r--  1 Daniel  staff   28840 May  5 13:59 genre_list_2020-05-05-13-59.csv
-rw-r--r--  1 Daniel  staff  954384 May  5 13:59 wiki-kaggle_genres_gender_cleaned_2020-05-05-13-59.csv
-rw-r--r--  1 Daniel  staff   28857 May  5 13:47 genre_list_2020-05-05-13-47.csv
-rw-r--r--  1 Daniel  staff  954392 May  5 13:47 wiki-kaggle_genres_gender_cleaned_2020-05-05-13-47.csv
-rw-r--r--  1 Daniel  staff   28941 May  5 13:43 genre_list_2020-05-05-13-43.csv
-rw-r--r--  1 Daniel  staff  954392 May  5 13:43 wiki-kaggle_genres_gender_cleaned_2020-05-05-13-43.csv
-rw-r--r--  1 Daniel  staff   28937 May  5 13:41 genre_list_202

## The code below here was run before the bands were removed. To reproduce the verification lists used, comment out the cells above that remove the bands and then rerun.

## Select 1% of artists to verify gender manually.

In [80]:
data.shape

(15497, 4)

In [81]:
data_male = data[data.gender == 'male']
data_female = data[data.gender == 'female']

In [82]:
tot = data.shape[0]
m = data_male.shape[0]
f = data_female.shape[0]
print('{} total artists'.format(tot))
print('{} female artists, or {:0.0f}%'.format(f, 100*f/(f+m)))
print('{} male artists, or {:0.0f}%'.format(m, 100*m/(f+m)))

15497 total artists
4869 female artists, or 31%
10628 male artists, or 69%


We will verify the same number of female as male artists. Let 
- $p$ be the fraction of artists to verify,
- $p_f$ be the fraction of female artists to verify
- $p_m$ be the fraction of male artists to verify

We want two conditions satisfied:

$$ p_f \cdot f = p_m \cdot m $$
$$ p_f \cdot f + p_m \cdot m = p \cdot (f+m)$$

These encode the requirements that
- we verify equal numbers of male and female artists
- the fractino of artists we verify is $p$

Solving for $p_f$ and $p_m$ leads to 

$$ p_f = \frac{p}{2f}(f+m)$$
$$ p_m = \frac{p}{2m}(f+m)$$

Define the functions for $p_f$ and $p_m$:

In [83]:
def pf(p,f,m):
    return p*(m+f)/(2*f)

def pm(p,f,m):
    return p*(m+f)/(2*m)

Substituting in $p=.01$, $f = 4864$, and $m = 10609$ leads to 

In [84]:
p = .01
p_f, p_m = pf(p,f,m), pm(p,f,m)

We select samples from the data:

In [85]:
sample_female = data_female.sample(frac = p_f)
sample_male = data_male.sample(frac = p_m)

In [86]:
sample_male.shape[0], sample_female.shape[0]

(77, 77)

In [87]:
sample_fm = pd.concat([sample_female,sample_male])

In [88]:
sample_size = sample_fm.shape[0]

Take a sample of the full size in order to shuffle:

In [89]:
sample_fm = sample_fm.sample(sample_size)

In [90]:
sample_fm.head()

,artist,gender,genrelist,genrelist_length
11137,Billy_Daniels,male,"[vocal_jazz, cabaret]",2
14246,Peter_Gifford,male,[rock],1
7259,Blanca,female,[contemporary_christian_music],1
16063,Whitfield_Crane,male,"[rock, hard_rock, heavy_metal]",3
19907,Sabrina_Johnston,female,"[dance, hip_hop]",2


In [91]:
tom_sample = sample_fm[:77]
dan_sample = sample_fm[77:]
tom_sample.shape, dan_sample.shape

((77, 4), (77, 4))

In [92]:
dan_sample.head(20)

,artist,gender,genrelist,genrelist_length
18163,Kim_Yarbrough,female,[r&b],1
2635,Lalaine,female,"[pop, alternative_rock, indie_rock]",3
18811,Brett_Gurewitz,male,"[punk_rock, hardcore_punk, melodic_hardcore, d...",4
8392,Dave_Flett,male,"[rock, progressive_rock]",2
3403,Julian_Taylor,male,[rock],1
1250,Francine_Reed,female,"[blues, jazz]",2
7106,Jewel,female,"[folk, pop, pop_rock, country]",4
19816,Kathy_Troccoli,female,"[contemporary_christian, inspirational, jazz]",3
6909,Barbara_Pittman,female,"[country, rockabilly]",2
9974,Matt_White,male,"[rock, pop]",2


Export only name and gender for verification:

In [103]:
tom_sample_verification = tom_sample[['artist','gender']]
tom_sample_verification.set_index(['artist'], inplace = True)

dan_sample_verification = dan_sample[['artist','gender']]
dan_sample_verification.set_index(['artist'], inplace = True)

In [104]:
tom_sample_verification.head()

,gender
artist,
Billy_Daniels,male
Peter_Gifford,male
Blanca,female
Whitfield_Crane,male
Sabrina_Johnston,female


In [105]:
dan_sample_verification.head()

,gender
artist,
Kim_Yarbrough,female
Lalaine,female
Brett_Gurewitz,male
Dave_Flett,male
Julian_Taylor,male


In [106]:
# tom_sample_verification.to_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/tom_sample_to_verify_{}.csv'.format(now))
# dan_sample_verification.to_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/dan_sample_to_verify_{}.csv'.format(now))

## Viewing the genre list:

In [122]:
glist = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/genre_list_2020-05-05-14-03.csv')

In [123]:
glist.drop(['Unnamed: 0'], axis =1, inplace = True)

In [124]:
glist = glist.sort_values('genre_list')

In [125]:
pd.set_option('display.max_rows', None)
pd.options.display.max_rows

In [1]:
#glist